In [ ]:
import pandas as pd
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from bs4 import BeautifulSoup
from pprint import pprint
import re
from collections import Counter
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [ ]:
#ошибочные домены, опечатки
mails = ('@gmail.com', '@gmail.ru', '@ya.ru', '@yandex.ru', '@mail.ru', '@bk.ru', '@rambler.ru', '@ro.ru', '@list.ru'
        ,'@bank24.ru', '@inbox.ru', '@icloud.com', '@indox.ru', '@me.com', '@e1.ru','@yahoo.com', '@outlook.com'
        ,'@ngs.ru', '@hotmail.com', '@live.ru', '@mail.com', '@mail.ri', '@protonmail.com', '@yandex.com', '@yndex.ru'
        ,'@gmai.com', '@mail.ry', '@mal.ru', '@yandex.ry', '@hey.com', '@bitrix24.ru', '@qmail.com', '@narod.ru'
        ,'@yanbex.ru', '@internet.ru', '@lenta.ru', '@saccount.biz', '@gmail.con', '@moedelo.org', '@gmal.com', '@yandeks.ru'
        ,'@live.com', '@yanex.ru', '@jmail.com', '@inbox.lv', '@meil.ru', '@live.com', '@ramler.ru', '@mail.r'
        ,'@yandex.r', '@gmail.co', '@yandex.ua', '@sibmail.com', '@mil.ru', '@mail.tu', '@my.com', '@icloud.ru'
        ,'@gmail.c', '@ayndex.ru', '@list.ry', '@inbox.com', '@yandex.tu', '@yangex.ru', '@maill.ru', '@gnail.com'
        ,'@ymail.com', '@ibox.ru', '@googlemail.com', '@hotmail.ru', '@eandex.ru', '@maii.ru', '@gmil.com', '@nail.ru'
        ,'@maul.ru', '@mail.u', '@msil.ru', '@list.tu', '@yandx.ru', '@email.ru','@gmeil.com', '@gmail.kom', '@ramber.ru'
        ,'@gmaill.com', '@yahoo.co.uk', '@mail.ua', '@gmsil.com', '@qip.ru', '@tandex.ru', '@yandex.by', '@jandex.ru'
        ,'@lnbox.ru', '@ynadex.ru', '@yansex.ru', '@myrambler.ru', '@bk.tu', '@yamdex.ru', '@gimail.com', '@andex.ru'
        ,'@vk.ru', '@yandax.ru', '@rambler.ua', '@yahdex.ru', '@yandekx.ru','@yandey.ru', '@gimal.com', '@ua.ru'
        ,'@ail.ru', '@bmail.ru', '@iclaud.com', '@ya.ry', '@yandeex.ru', '@nextmail.ru', '@email.com', '@cloud.com'
        ,'@ramblel.ru', '@yandexs.ru', '@gmail.om', '@maiil.ru', '@inbox.ry', '@ysndex.ru', '@gamail.com', '@ma.il.ru'
        ,'@gail.com', '@dmail.com', '@gmaiil.com', '@yandeх.ru', '@yandrx.ru', '@gmailc.om', '@ramdler.ru', '@yundex.ru'
        ,'@iclud.com', '@gmail.cm', '@gmali.com', '@y.ru', '@inbox.r', '@imail.ru', '@rabler.ru', '@imbox.ru', '@unbox.ru'
        ,'@mail.eu', '@yanfex.ru', '@mial.ru', '@mall.ru', '@yadex.ru', '@mmail.ru', '@mail.magicwebsolutions.co.uk')

In [ ]:
#делаем запрос в БД
request = """
select * 
from contacts c
where type='e-mail'
"""
df = pd.read_sql_query(request, dwh)

In [ ]:
#или можем прочитать csv
df = pd.read_csv('emails.csv', sep = ';', encoding='utf8', engine='python')

In [ ]:
domens = {}

#при учёте, что у нас есть следующие колонки:
#email - адрес электронной почты
#id - id организации
for row in df.to_dict('records'):
    if '@' in row['email']:
        if row['email'].lower().strip().endswith(mails)==False:
            domen = row['email'].split('@')[1].strip().lower()
            if 'mail' not in domen:
                if row['id'] not in domens:
                    domens.update({row['id']:[domen]})
                elif row['id'] in domens and domen not in domens[row['id']]:                                    
                    domens[row['id']].append(domen)

In [ ]:
#смотрим, какой домен сколько раз встречается
count = []
for i in domens.values():
    for j in i[0]:
        count.append(j)
Counter(count).most_common()

In [ ]:
#посмотреть организации с определённым доменом
domen = 'mai.ru'
for i in domens.values():
    if domen in i[0]:
        print(i)

In [ ]:
#проверяем на доступность домена - сайта
real_domens = []
head={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}

for domen in tqdm(range(len(domens))):
    it = domen
    id_c = list(domens)[domen]
    id_domens = domens[list(domens)[domen]]
    for domen in id_domens: 
        try:
            resp = requests.get(f'https://{domen}', headers=head, verify=False, timeout=10)
            #перенаправление на актуальный адрес
            location = resp.url
            #заголовок
            try:
                soup = BeautifulSoup(resp.text, 'lxml')
                title = soup.find_all('title')[0].text.replace('\xa0', ' ')
                if 'Ð' in title:
                    title = title.encode('latin1').decode('utf8')
                crakozbr = ['ä','û','ñ','ï','õ','ò','à','â','è','ø','î','á','ð','ü','ó','æ','å','ê','ã','í','ÿ','ì']
                if any(e in crakozbr for e in title.lower()):
                    title = title.encode('windows-1252').decode('cp1251', errors='ignore')
            except: title = ''
            real_domens.append([id, domen, location, resp.status_code, ' '.join(title.split())])
        except:
            try:
                resp = requests.get(f'http://{domen}', headers=head, verify=False, timeout=10)
                #перенаправление на актуальный адрес
                location = resp.url
                #заголовок
                try:
                    soup = BeautifulSoup(resp.text, 'lxml')
                    title = soup.find_all('title')[0].text
                    if 'Ð' in title:
                        title = title.encode('latin1').decode('utf8')
                except: title = ''
                real_domens.append([id, domen, location, resp.status_code, ' '.join(title.split())])
            except: 
                pass

In [ ]:
domens_shop = ['reg.ru', 'продается домен', 'домен не найден', 'ru-center', 'домен припаркован', 'домен отключен'
              ,'ru-tld.ru', 'домен зарегистрирован через «джино»', 'домен продается', 'домен не найден', 'домен не добавлен'
              ,'срок регистрации домена закончился', 'домен не прилинкован', 'доменное имя временно заблокировано'
              ,'истёк срок регистрации домена', 'срок подключения домена истёк', 'перенаправление домена не осуществляется'
              ,'домен временно недоступен', 'присоединить домен заново', 'информация о домене', 'начальная страница домена'
              ,'в список поддерживаемых системой управления сайтом', 'домен еще не подключен к вашему сайту'
              ,'- домен зарегистрирован', 'срок регистрации домена', 'доступ запрещен', 'доступ к сайту невозможен'
              ,'в доступе на страницу отказано', 'в доступе на данную страницу отказано', '403 - ошибка доступа'
              ,'сервис недоступен / service unavailable', 'ошибка 403. доступ запрещен', 'сайт недоступен'
              ,'сайт в настоящее время недоступен', 'доступ к сайту временно закрыт', 'в доступе на страницу отказано'
              ,'403', '404', 'доступ к сайту приостановлен', 'хостинг-провайдер timeweb.ru | мы рады приветствовать вас среди наших клиентов!']

not_available = []
for i in real_domens.copy():
    if 'домен' in i[4].lower() and ('продается' in i[4].lower() or 'продаётся' in i[4].lower()):
        real_domens.remove(i)
    elif len([word for word in domens_shop if word in i[4].lower()])>0:
        real_domens.remove(i)
    elif len([word for word in mails if word[1:] in i[1].lower()])>0:
        real_domens.remove(i)

for i in real_domens.copy():
    if i[4] in not_available or i[3] in (400, 401, 404, 403, 402, 412, 502, 503, 501
                                        ,520, 521, 523, 525, 499, 426, 423, 451, 999
                                        ,428, 405, 410):
        real_domens.remove(i)#не уверен, что здесь лучше удалять, а не переносить

In [ ]:
#пишем в датафрейм и csv
dfcsv = pd.DataFrame(real_domens, columns=['id', 'domen', 'location', 'status_code', 'title'])
dfcsv.to_csv('сайты.csv', index=False)